In [1]:
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install mdatagen

In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from geomloss import SamplesLoss

import ot

import os
import pickle as pkl
import copy

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from utils import *
from SoftImpute import softimpute, cv_softimpute
#from DataSetLoader import dataset_loader, ground_truth
from SinkhornImputation import SinkhornImputation
from RR_imputer import RRimputer
import matplotlib.pyplot as plt
from Inject_Missing_Values import *


import argparse
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
import pandas as pd

torch.set_default_tensor_type('torch.DoubleTensor')

c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


Testing the new Missing function

In [2]:
d =  {
    "A": [10,10,20,40,70,40,50,80,10,40,20,60,50,30,20],
    "B": [1,1,2,4,7,4,5,8,1,4,2,6,5,3,2],
    "C":[15,15,30,60,105,60,75,120,15,60,30,90,75,45,30],
    "D":[45,30,60,32,58,63,25,55,46,78,92,60,34,27,35]

}
data = pd.DataFrame(d)

In [3]:
print(data)

     A  B    C   D
0   10  1   15  45
1   10  1   15  30
2   20  2   30  60
3   40  4   60  32
4   70  7  105  58
5   40  4   60  63
6   50  5   75  25
7   80  8  120  55
8   10  1   15  46
9   40  4   60  78
10  20  2   30  92
11  60  6   90  60
12  50  5   75  34
13  30  3   45  27
14  20  2   30  35


In [4]:
generator = Inject_Missing_Values()

MCAR

In [21]:
selected_columns = ['A','B']
miss_mcar,index_mcar = generator.MCAR(data,missing_rate=10)
print(miss_mcar.isna().sum())


print(miss_mcar)
print(miss_mcar.notna().sum())
print(index_mcar)

A    1
B    0
C    3
D    2
dtype: int64
       A  B      C     D
0    NaN  1    NaN  45.0
1   10.0  1   15.0  30.0
2   20.0  2   30.0  60.0
3   40.0  4    NaN  32.0
4   70.0  7  105.0  58.0
5   40.0  4   60.0  63.0
6   50.0  5   75.0  25.0
7   80.0  8  120.0  55.0
8   10.0  1   15.0  46.0
9   40.0  4   60.0  78.0
10  20.0  2   30.0  92.0
11  60.0  6   90.0   NaN
12  50.0  5    NaN  34.0
13  30.0  3   45.0  27.0
14  20.0  2   30.0   NaN
A    14
B    15
C    12
D    13
dtype: int64
{'14,1': 'MCAR', '10,3': 'MCAR', '9,1': 'MCAR', '1,2': 'MCAR', '5,0': 'MCAR', '2,3': 'MCAR', '12,2': 'MCAR', '9,0': 'MCAR', '10,2': 'MCAR', '2,1': 'MCAR', '13,3': 'MCAR', '7,0': 'MCAR', '5,1': 'MCAR', '1,3': 'MCAR', '8,1': 'MCAR', '11,1': 'MCAR', '5,2': 'MCAR', '4,1': 'MCAR', '3,2': 'MCAR', '11,0': 'MCAR', '2,0': 'MCAR', '11,2': 'MCAR', '6,0': 'MCAR', '0,2': 'MCAR', '4,2': 'MCAR', '1,0': 'MCAR', '3,0': 'MCAR', '0,0': 'MCAR', '14,2': 'MCAR', '3,3': 'MCAR', '14,0': 'MCAR', '12,0': 'MCAR', '13,2': 'MCAR', '6,2':

In [22]:
missing_percentage = (miss_mcar.isnull().sum() / len(miss_mcar)) * 100
print(missing_percentage)


A     6.666667
B     0.000000
C    20.000000
D    13.333333
dtype: float64


In [23]:
total_missing_percentage = miss_mcar.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")


Total Missing Percentage: 10.00%


MAR


In [24]:


dependencies_MAR = {
    "A": {
        "influencers": ["C","B"],
        "condition": lambda row: row["C"]+row["B"] > 30  # Just a boolean condition
          # Separate probability function
    },
    "B": {
        "influencers": ["C"],
        "condition": lambda row: True,
        "probability": lambda row: 0.2 if row["C"] < 30 else 0.1
    }
}

In [30]:
miss_mar,index_mar = generator.MAR(miss_mcar,dependencies_MAR,missing_rate=15)

In [26]:
print(index_mar)

{'14,1': 'MCAR', '10,3': 'MCAR', '9,1': 'MCAR', '1,2': 'MCAR', '5,0': 'MAR,MCAR', '2,3': 'MCAR', '12,2': 'MCAR', '9,0': 'MAR,MCAR', '10,2': 'MCAR', '2,1': 'MCAR', '13,3': 'MCAR', '7,0': 'MAR,MCAR', '5,1': 'MCAR', '1,3': 'MCAR', '8,1': 'MCAR', '11,1': 'MCAR', '5,2': 'MCAR', '4,1': 'MCAR', '3,2': 'MCAR', '11,0': 'MAR,MCAR', '2,0': 'MAR,MCAR', '11,2': 'MCAR', '6,0': 'MAR,MCAR', '0,2': 'MCAR', '4,2': 'MCAR', '1,0': 'MCAR', '3,0': 'MCAR', '0,0': 'MCAR', '14,2': 'MCAR', '3,3': 'MCAR', '14,0': 'MAR,MCAR', '12,0': 'MCAR', '13,2': 'MCAR', '6,2': 'MCAR', '13,0': 'MAR,MCAR', '7,3': 'MCAR', '7,2': 'MCAR', '3,1': 'MCAR', '6,3': 'MCAR', '2,2': 'MCAR', '8,3': 'MCAR', '13,1': 'MCAR', '1,1': 'MCAR', '10,0': 'MAR,MCAR', '11,3': 'MCAR', '14,3': 'MCAR'}


In [31]:
print(miss_mar)

       A    B      C     D
0    NaN  1.0    NaN  45.0
1   10.0  1.0   15.0  30.0
2    NaN  2.0   30.0  60.0
3   40.0  4.0    NaN  32.0
4    NaN  7.0  105.0  58.0
5   40.0  4.0   60.0  63.0
6    NaN  NaN   75.0  25.0
7   80.0  8.0  120.0  55.0
8   10.0  1.0   15.0  46.0
9   40.0  4.0   60.0  78.0
10   NaN  2.0   30.0  92.0
11   NaN  NaN   90.0   NaN
12  50.0  5.0    NaN  34.0
13   NaN  3.0   45.0  27.0
14   NaN  2.0   30.0   NaN


In [32]:
total_missing_percentage = miss_mar.isnull().sum().sum() / miss_mar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")

Total Missing Percentage: 25.00%


MNAR

In [33]:
dependencies_MNAR = {
    "D": {
        "condition": lambda row: True,
        "probability": lambda row: 0.3 if row["D"] < 30 else 0.1  # High earners are more likely to not report salary
    },
    "C": {
        "condition": lambda row: row["C"] > 50, # High BP patients may avoid reporting BP readings
        "probability": lambda row: 0.2 
    },
    
}

In [43]:
miss_mnar,index_mnar = generator.MNAR(data,dependencies_MNAR,missing_rate=10)

In [44]:
print(miss_mnar)

     A  B      C     D
0   10  1   15.0  45.0
1   10  1   15.0  30.0
2   20  2   30.0   NaN
3   40  4    NaN  32.0
4   70  7  105.0  58.0
5   40  4    NaN  63.0
6   50  5    NaN  25.0
7   80  8  120.0  55.0
8   10  1   15.0  46.0
9   40  4    NaN  78.0
10  20  2   30.0   NaN
11  60  6   90.0  60.0
12  50  5   75.0  34.0
13  30  3   45.0  27.0
14  20  2   30.0  35.0


In [46]:
print(index_mnar)

{'14,1': 'MCAR', '10,3': 'MNAR,MCAR', '9,1': 'MCAR', '1,2': 'MCAR', '5,0': 'MAR,MCAR', '2,3': 'MNAR,MCAR', '12,2': 'MNAR,MCAR', '9,0': 'MAR,MCAR', '10,2': 'MCAR', '2,1': 'MCAR', '13,3': 'MNAR,MCAR', '7,0': 'MAR,MCAR', '5,1': 'MCAR', '1,3': 'MCAR', '8,1': 'MCAR', '11,1': 'MAR,MCAR', '5,2': 'MNAR,MCAR', '4,1': 'MCAR', '3,2': 'MNAR,MCAR', '11,0': 'MAR,MNAR,MCAR', '2,0': 'MAR,MCAR', '11,2': 'MNAR,MCAR', '6,0': 'MAR,MNAR,MCAR', '0,2': 'MCAR', '4,2': 'MNAR,MCAR', '1,0': 'MCAR', '3,0': 'MCAR', '0,0': 'MCAR', '14,2': 'MCAR', '3,3': 'MNAR,MCAR', '14,0': 'MAR,MCAR', '12,0': 'MCAR', '13,2': 'MCAR', '6,2': 'MNAR,MCAR', '13,0': 'MAR,MCAR', '7,3': 'MCAR', '7,2': 'MNAR,MCAR', '3,1': 'MCAR', '6,3': 'MNAR,MCAR', '2,2': 'MCAR', '8,3': 'MNAR,MCAR', '13,1': 'MCAR', '1,1': 'MCAR', '10,0': 'MAR,MCAR', '11,3': 'MNAR,MCAR', '14,3': 'MCAR', '12,1': 'MAR', '6,1': 'MAR', '4,0': 'MAR', '9,2': 'MNAR', '9,3': 'MNAR', '0,3': 'MNAR'}


In [45]:
total_missing_percentage = miss_mnar.isnull().sum().sum() / miss_mnar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")

Total Missing Percentage: 10.00%
